<a href="https://colab.research.google.com/github/xniu3/Gitter/blob/master/ECE625_project_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Necessary data processing and machine learning tools and packages. 

In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error,r2_score

2. Mount the path to the Google drive. 

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
!cd drive

3. Read and clean the dataset. As the dataset is already devided into traning and testing dataset, we could directly clean and process them seperately without splitting. 

In [103]:
EstateTrainData = pd.read_csv("/content/drive/MyDrive/EdmontonRealEstateData_train.csv",encoding = "ISO-8859-1")

In [104]:
EstateTestData = pd.read_csv("/content/drive/MyDrive/EdmontonRealEstateData_test.csv",encoding = "ISO-8859-1")

In [105]:
EstateTrainData.shape

(42779, 33)

In [106]:
EstateTestData.shape

(10695, 33)

Seems like the first column is the original sequence of a big dataset, which is composed of the training dataset and the testing dataset. 

In [107]:
EstateTrainData.info

<bound method DataFrame.info of        Unnamed: 0  taxroll_number          landuse_description  \
0               1        10195786  100% SINGLE FAMILY DWELLING   
1               2        10195785  100% SINGLE FAMILY DWELLING   
2               3        10195784  100% SINGLE FAMILY DWELLING   
3               4        10195783  100% SINGLE FAMILY DWELLING   
4               7        10195780  100% SINGLE FAMILY DWELLING   
...           ...             ...                          ...   
42774       71461        10266550  100% SINGLE FAMILY DWELLING   
42775       71463         9970596  100% SINGLE FAMILY DWELLING   
42776       71466         9970598  100% SINGLE FAMILY DWELLING   
42777       71471         9952730  100% SINGLE FAMILY DWELLING   
42778       71499        10228415  100% SINGLE FAMILY DWELLING   

        market_building_class  effective_build_year  net_area  \
0      2 STOREY WITH BASEMENT                2011.0    188.50   
1      2 STOREY WITH BASEMENT                

In [108]:
EstateTestData.info

<bound method DataFrame.info of        Unnamed: 0  taxroll_number          landuse_description  \
0               5        10195782  100% SINGLE FAMILY DWELLING   
1               6        10195781  100% SINGLE FAMILY DWELLING   
2               8        10195779  100% SINGLE FAMILY DWELLING   
3               9        10195778  100% SINGLE FAMILY DWELLING   
4              10        10195777  100% SINGLE FAMILY DWELLING   
...           ...             ...                          ...   
10690       71465         9970597  100% SINGLE FAMILY DWELLING   
10691       71470         9952728  100% SINGLE FAMILY DWELLING   
10692       71472         9952732  100% SINGLE FAMILY DWELLING   
10693       71474         9952733  100% SINGLE FAMILY DWELLING   
10694       71475         9952734  100% SINGLE FAMILY DWELLING   

                      market_building_class  effective_build_year  net_area  \
0                    2 STOREY WITH BASEMENT                2010.0    208.30   
1                

First, we clear the unrelated parameters and keep 18 useful and correlated parameters shown as follows. 

In [109]:
EstateTrainData = EstateTrainData.drop(['house_suit','house_suff','street_name','postal_code','city',\
'full_address','landuse_description','geometry','valuation_group','neighbourhood','build_year_mbc',\
'display_type','lot_size','house_number','market_building_class'],axis=1)
EstateTestData = EstateTestData.drop(['house_suit','house_suff','street_name','postal_code','city',\
'full_address','landuse_description','geometry','valuation_group','neighbourhood','build_year_mbc',\
'display_type','lot_size','house_number','market_building_class'],axis=1)

Next, we clean the dataset by evicting some samples with values out-of-boundary, as well as some samples with value not applicable or "N.A.". 

In [110]:
EstateTrainData = EstateTrainData.loc[EstateTrainData.loc[:,'net_area']>0,:]
EstateTrainData = EstateTrainData.loc[EstateTrainData.loc[:,'effective_build_year'] != '',:]
EstateTrainData = EstateTrainData.loc[EstateTrainData.loc[:,'assessed_value'] != '0',:]
EstateTrainData = EstateTrainData.dropna()
EstateTrainData.shape

(40451, 18)

In [111]:
EstateTrainData

,Unnamed: 0,taxroll_number,effective_build_year,net_area,basement_finished,has_garage,has_fireplace,assessed_value,fully_taxable,fully_complete,building_count,walkout_basement,air_conditioning,site_coverage,tot_gross_area_description,result_code,lon,lat
0,1,10195786,2011.0,188.50,Yes,Yes,Yes,470500,Yes,Yes,1,NO,NO,25%,227.5 M2 / 2448.8 FT2,1110,-113.430993,53.424599
1,2,10195785,2011.0,178.00,NO,Yes,Yes,442500,Yes,Yes,1,NO,NO,29%,215.6 M2 / 2320.7 FT2,1110,-113.431231,53.424560
2,3,10195784,2011.0,178.30,NO,Yes,Yes,448000,Yes,Yes,1,NO,NO,31%,217.1 M2 / 2336.8 FT2,1110,-113.431416,53.424549
3,4,10195783,2010.0,182.80,NO,Yes,Yes,447500,Yes,Yes,1,NO,NO,33%,226 M2 / 2432.6 FT2,1110,-113.431579,53.424548
4,7,10195780,2010.0,199.90,NO,Yes,Yes,462000,Yes,Yes,1,NO,NO,32%,238.9 M2 / 2571.5 FT2,1110,-113.432071,53.424541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42774,71461,10266550,2013.0,140.50,NO,NO,NO,379000,Yes,Yes,1,NO,NO,24%,140.5 M2 / 1512.3 FT2,1110,-113.696045,53.538105
42775,71463,9970596,2005.0,173.46,NO,Yes,Yes,725500,Yes,Yes,1,NO,NO,2%,269.76 M2 / 2903.7 FT2,1110,-113.325837,53.673328
42776,71466,9970598,2003.0,259.50,Yes,Yes,Yes,1012000,Yes,Yes,1,Yes,NO,4%,397.5 M2 / 4278.7 FT2,1110,-113.325541,53.674791
42777,71471,9952730,2001.0,153.60,Yes,Yes,Yes,772000,Yes,Yes,1,NO,NO,2%,332 M2 / 3573.6 FT2,1110,-113.323203,53.679244


Only 40451 out of 42279 data samples are kept. Then, we clean the testing dataset. 

In [112]:
EstateTestData = EstateTestData.loc[EstateTestData.loc[:,'net_area']>0,:]
EstateTestData = EstateTestData.loc[EstateTestData.loc[:,'effective_build_year'] != '',:]
EstateTestData = EstateTestData.loc[EstateTestData.loc[:,'assessed_value'] != '0',:]
EstateTestData = EstateTestData.dropna()
EstateTestData.shape

(10064, 18)

In [113]:
EstateTestData

,Unnamed: 0,taxroll_number,effective_build_year,net_area,basement_finished,has_garage,has_fireplace,assessed_value,fully_taxable,fully_complete,building_count,walkout_basement,air_conditioning,site_coverage,tot_gross_area_description,result_code,lon,lat
0,5,10195782,2010.0,208.30,NO,Yes,Yes,469500,Yes,Yes,1,NO,NO,34%,246.7 M2 / 2655.5 FT2,1110,-113.431742,53.424544
1,6,10195781,2010.0,164.30,Yes,Yes,Yes,423000,Yes,Yes,1,NO,NO,29%,199.3 M2 / 2145.2 FT2,1110,-113.431905,53.424545
2,8,10195779,2010.0,170.20,NO,Yes,Yes,433500,Yes,Yes,1,NO,NO,29%,209 M2 / 2249.7 FT2,1110,-113.432237,53.424542
3,9,10195778,2011.0,168.20,NO,Yes,Yes,435500,Yes,Yes,1,NO,NO,29%,205.8 M2 / 2215.2 FT2,1110,-113.432399,53.424536
4,10,10195777,2010.0,208.20,NO,Yes,Yes,468000,Yes,Yes,1,NO,NO,35%,246.6 M2 / 2654.4 FT2,1110,-113.432562,53.424535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10690,71465,9970597,2002.0,140.06,Yes,Yes,Yes,643000,Yes,Yes,1,NO,NO,2%,192.99 M2 / 2077.3 FT2,1110,-113.325695,53.674033
10691,71470,9952728,1992.0,110.88,NO,Yes,NO,502500,Yes,Yes,2,NO,NO,0%,231.313 M2 / 2489.8 FT2,1110,-113.323809,53.678443
10692,71472,9952732,1998.0,169.60,Yes,Yes,Yes,788500,Yes,Yes,1,NO,NO,2%,256.7 M2 / 2763.1 FT2,1110,-113.322486,53.679981
10693,71474,9952733,1998.0,253.60,NO,Yes,Yes,752000,Yes,Yes,1,NO,NO,2%,334.4 M2 / 3599.5 FT2,1110,-113.321340,53.680806


Only 10064 out of 10695 data samples are kept. 

In order to ease our data processing, we convert some string value (like "NO" or "YES") to integer or boolean value shown as follows. 

In [114]:
EstateTrainData.loc[EstateTrainData['basement_finished']=='NO','basement_finished']=0
EstateTrainData.loc[EstateTrainData['basement_finished']=='Yes','basement_finished']=1
EstateTrainData['basement_finished']=EstateTrainData['basement_finished'].astype('int')

EstateTrainData.loc[EstateTrainData['has_garage']=='NO','has_garage']=0
EstateTrainData.loc[EstateTrainData['has_garage']=='Yes','has_garage']=1
EstateTrainData['has_garage']=EstateTrainData['has_garage'].astype('int')

EstateTrainData.loc[EstateTrainData['has_fireplace']=='NO','has_fireplace']=0
EstateTrainData.loc[EstateTrainData['has_fireplace']=='Yes','has_fireplace']=1
EstateTrainData['has_fireplace']=EstateTrainData['has_fireplace'].astype('int')

EstateTrainData.loc[EstateTrainData['fully_taxable']=='NO','fully_taxable']=0
EstateTrainData.loc[EstateTrainData['fully_taxable']=='Yes','fully_taxable']=1
EstateTrainData['fully_taxable']=EstateTrainData['fully_taxable'].astype('int')

EstateTrainData.loc[EstateTrainData['fully_complete']=='NO','fully_complete']=0
EstateTrainData.loc[EstateTrainData['fully_complete']=='Yes','fully_complete']=1
EstateTrainData['fully_complete']=EstateTrainData['fully_complete'].astype('int')

EstateTrainData.loc[EstateTrainData['walkout_basement']=='NO','walkout_basement']=0
EstateTrainData.loc[EstateTrainData['walkout_basement']=='Yes','walkout_basement']=1
EstateTrainData['walkout_basement']=EstateTrainData['walkout_basement'].astype('int')

EstateTrainData.loc[EstateTrainData['air_conditioning']=='NO','air_conditioning']=0
EstateTrainData.loc[EstateTrainData['air_conditioning']=='Yes','air_conditioning']=1
EstateTrainData['air_conditioning']=EstateTrainData['air_conditioning'].astype('int')

m = EstateTrainData['site_coverage'].str.contains('\d+%')
EstateTrainData['site_coverage'] = \
    EstateTrainData['site_coverage'].str.extract('(\d+)', expand=False).astype(float)
EstateTrainData.loc[m, 'site_coverage'] *= 0.01

m = EstateTrainData['tot_gross_area_description'].str.contains('\d+.')
EstateTrainData['tot_gross_area_description'] = \
    EstateTrainData['tot_gross_area_description'].str.extract('(\d+)', expand=False).astype(float)
EstateTrainData.loc[m, 'tot_gross_area_description'] *= 1

EstateTrainData['assessed_value'] = EstateTrainData['assessed_value'].astype('float')
EstateTrainData['effective_build_year'] = EstateTrainData['effective_build_year'].astype('int')
EstateTrainData['building_count'] = EstateTrainData['building_count'].astype('int')


In [115]:
EstateTrainData

,Unnamed: 0,taxroll_number,effective_build_year,net_area,basement_finished,has_garage,has_fireplace,assessed_value,fully_taxable,fully_complete,building_count,walkout_basement,air_conditioning,site_coverage,tot_gross_area_description,result_code,lon,lat
0,1,10195786,2011,188.50,1,1,1,470500.0,1,1,1,0,0,0.25,227.0,1110,-113.430993,53.424599
1,2,10195785,2011,178.00,0,1,1,442500.0,1,1,1,0,0,0.29,215.0,1110,-113.431231,53.424560
2,3,10195784,2011,178.30,0,1,1,448000.0,1,1,1,0,0,0.31,217.0,1110,-113.431416,53.424549
3,4,10195783,2010,182.80,0,1,1,447500.0,1,1,1,0,0,0.33,226.0,1110,-113.431579,53.424548
4,7,10195780,2010,199.90,0,1,1,462000.0,1,1,1,0,0,0.32,238.0,1110,-113.432071,53.424541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42774,71461,10266550,2013,140.50,0,0,0,379000.0,1,1,1,0,0,0.24,140.0,1110,-113.696045,53.538105
42775,71463,9970596,2005,173.46,0,1,1,725500.0,1,1,1,0,0,0.02,269.0,1110,-113.325837,53.673328
42776,71466,9970598,2003,259.50,1,1,1,1012000.0,1,1,1,1,0,0.04,397.0,1110,-113.325541,53.674791
42777,71471,9952730,2001,153.60,1,1,1,772000.0,1,1,1,0,0,0.02,332.0,1110,-113.323203,53.679244


Same process for testing dataset. 

In [116]:
EstateTestData.loc[EstateTestData['basement_finished']=='NO','basement_finished']=0
EstateTestData.loc[EstateTestData['basement_finished']=='Yes','basement_finished']=1
EstateTestData['basement_finished']=EstateTestData['basement_finished'].astype('int')

EstateTestData.loc[EstateTestData['has_garage']=='NO','has_garage']=0
EstateTestData.loc[EstateTestData['has_garage']=='Yes','has_garage']=1
EstateTestData['has_garage']=EstateTestData['has_garage'].astype('int')

EstateTestData.loc[EstateTestData['has_fireplace']=='NO','has_fireplace']=0
EstateTestData.loc[EstateTestData['has_fireplace']=='Yes','has_fireplace']=1
EstateTestData['has_fireplace']=EstateTestData['has_fireplace'].astype('int')

EstateTestData.loc[EstateTestData['fully_taxable']=='NO','fully_taxable']=0
EstateTestData.loc[EstateTestData['fully_taxable']=='Yes','fully_taxable']=1
EstateTestData['fully_taxable']=EstateTestData['fully_taxable'].astype('int')

EstateTestData.loc[EstateTestData['fully_complete']=='NO','fully_complete']=0
EstateTestData.loc[EstateTestData['fully_complete']=='Yes','fully_complete']=1
EstateTestData['fully_complete']=EstateTestData['fully_complete'].astype('int')

EstateTestData.loc[EstateTestData['walkout_basement']=='NO','walkout_basement']=0
EstateTestData.loc[EstateTestData['walkout_basement']=='Yes','walkout_basement']=1
EstateTestData['walkout_basement']=EstateTestData['walkout_basement'].astype('int')

EstateTestData.loc[EstateTestData['air_conditioning']=='NO','air_conditioning']=0
EstateTestData.loc[EstateTestData['air_conditioning']=='Yes','air_conditioning']=1
EstateTestData['air_conditioning']=EstateTestData['air_conditioning'].astype('int')

m = EstateTestData['site_coverage'].str.contains('\d+%')
EstateTestData['site_coverage'] = \
    EstateTestData['site_coverage'].str.extract('(\d+)', expand=False).astype(float)
EstateTestData.loc[m, 'site_coverage'] *= 0.01

m = EstateTestData['tot_gross_area_description'].str.contains('\d+.')
EstateTestData['tot_gross_area_description'] = \
    EstateTestData['tot_gross_area_description'].str.extract('(\d+)', expand=False).astype(float)
EstateTestData.loc[m, 'tot_gross_area_description'] *= 1

EstateTestData['assessed_value'] = EstateTestData['assessed_value'].astype('float')
EstateTestData['effective_build_year'] = EstateTestData['effective_build_year'].astype('int')
EstateTestData['building_count'] = EstateTestData['building_count'].astype('int')

In [117]:
EstateTestData

,Unnamed: 0,taxroll_number,effective_build_year,net_area,basement_finished,has_garage,has_fireplace,assessed_value,fully_taxable,fully_complete,building_count,walkout_basement,air_conditioning,site_coverage,tot_gross_area_description,result_code,lon,lat
0,5,10195782,2010,208.30,0,1,1,469500.0,1,1,1,0,0,0.34,246.0,1110,-113.431742,53.424544
1,6,10195781,2010,164.30,1,1,1,423000.0,1,1,1,0,0,0.29,199.0,1110,-113.431905,53.424545
2,8,10195779,2010,170.20,0,1,1,433500.0,1,1,1,0,0,0.29,209.0,1110,-113.432237,53.424542
3,9,10195778,2011,168.20,0,1,1,435500.0,1,1,1,0,0,0.29,205.0,1110,-113.432399,53.424536
4,10,10195777,2010,208.20,0,1,1,468000.0,1,1,1,0,0,0.35,246.0,1110,-113.432562,53.424535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10690,71465,9970597,2002,140.06,1,1,1,643000.0,1,1,1,0,0,0.02,192.0,1110,-113.325695,53.674033
10691,71470,9952728,1992,110.88,0,1,0,502500.0,1,1,2,0,0,0.00,231.0,1110,-113.323809,53.678443
10692,71472,9952732,1998,169.60,1,1,1,788500.0,1,1,1,0,0,0.02,256.0,1110,-113.322486,53.679981
10693,71474,9952733,1998,253.60,0,1,1,752000.0,1,1,1,0,0,0.02,334.0,1110,-113.321340,53.680806


In [118]:
!pip install --upgrade category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [120]:
EstateTrainData.to_csv('ClearedTrainData.csv')

In [121]:
EstateTestData.to_csv('ClearedTestData.csv')

In [122]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectFpr

In [132]:
X_train = EstateTrainData.loc[:,['effective_build_year','net_area','basement_finished',\
                           'has_garage','has_fireplace','fully_taxable','fully_complete',\
                         'building_count','walkout_basement','air_conditioning','site_coverage',\
                         'tot_gross_area_description','lon','lat']]
Y_train = EstateTrainData['assessed_value']

In [133]:
X_test = EstateTestData.loc[:,['effective_build_year','net_area','basement_finished',\
                           'has_garage','has_fireplace','fully_taxable','fully_complete',\
                         'building_count','walkout_basement','air_conditioning','site_coverage',\
                         'tot_gross_area_description','lon','lat']]
Y_test = EstateTestData['assessed_value']

In [134]:
F_test,p_value=f_regression(X_train,Y_train)
pd.DataFrame({
              'F_test':F_test,'p_value':p_value})

,F_test,p_value
0,1648.28,0.00
1,55276.96,0.00
2,764.70,0.00
3,1159.62,0.00
4,4891.60,0.00
5,6.94,0.01
6,87.55,0.00
7,25.90,0.00
8,5443.19,0.00
9,3469.04,0.00


In [150]:
result = pd.DataFrame(columns=('Linear Regression','Ridge Regression','LASSO','RFR','SVR'), \
                      index = ["MAPE", 'MSE', 'r2 score'])

### Linear Regression

In [139]:
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [140]:
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

(40451, 14) (40451,) (10064, 14) (10064,)


In [151]:
LinearReg = LinearRegression()
LinearReg.fit(X_train,Y_train)

print(LinearReg.coef_)

Linear_predict = LinearReg.predict(X_test)
Linear_mape = mean_absolute_percentage_error(Y_test,Linear_predict)
Linear_mse = mean_squared_error(Y_test,Linear_predict)
Linear_r2 = r2_score(Y_test,Linear_predict)
print('Mean Absolute Percentage Error for Linear regression is: ', Linear_mape)
print('Mean Squared Error for Linear regression is: ', sqrt(Linear_mse))
print('R2 score for Linear regression is: ', Linear_r2)

result['Linear Regression'] = [Linear_mape,sqrt(Linear_mse),Linear_r2]

[-3.91913447e+02  5.29604153e+02  5.98538555e+04 -7.19817192e+04
 -7.53012501e+02  8.65845210e+04  1.80596781e+05 -5.75634206e+05
  1.56761755e+05  9.51525947e+04 -1.39233276e+05  2.13281588e+03
 -1.35274168e+05 -1.42893979e+05]
Mean Absolute Percentage Error for Linear regression is:  0.1424543780085256
Mean Squared Error for Linear regression is:  157998.4283306371
R2 score for Linear regression is:  0.6805160747102332


## Ridge Regression

In [144]:
from sklearn import linear_model

In [145]:
RidgeReg = linear_model.Ridge(alpha = 1.0)
RidgeReg.fit(X_train,Y_train)
Ridge_predict = RidgeReg.predict(X_test)

Ridge_mape = mean_absolute_percentage_error(Y_test,Ridge_predict)
Ridge_mse = mean_squared_error(Y_test,Ridge_predict)
Ridge_r2 = r2_score(Y_test,Ridge_predict)
print('Mean Absolute Percentage Error for Ridge regression is: ',Ridge_mape)
print('Mean Squared Error for Ridge regression is: ', sqrt(Ridge_mse))
print('R2 score for Ridge regression is: ', Ridge_r2)

Mean Absolute Percentage Error for Ridge regression is:  0.14239510054179869
Mean Squared Error for Ridge regression is:  157914.6838748835
R2 score for Ridge regression is:  0.6808546593050707


In [152]:
result['Ridge Regression'] = [Ridge_mape, sqrt(Ridge_mse), Ridge_r2]

## LASSO

In [153]:
Lasso = linear_model.Lasso(alpha = 0.1)
Lasso.fit(X_train,Y_train)

Lasso_predict = Lasso.predict(X_test)
Lasso_mape = mean_absolute_percentage_error(Y_test,Lasso_predict)
Lasso_mse = mean_squared_error(Y_test,Lasso_predict)
Lasso_r2 = r2_score(Y_test,Lasso_predict)
print('Mean Absolute Percentage Error for LASSO is: ',Lasso_mape)
print('Mean Squared Error for LASSO is: ', sqrt(Lasso_mse))
print('R2 score for Lasso is: ', Lasso_r2)

Mean Absolute Percentage Error for LASSO is:  0.14245334117312672
Mean Squared Error for LASSO is:  157997.79320937034
r2 score for Lasso is:  0.6805186432247388


In [154]:
result['LASSO'] = [Lasso_mape, sqrt(Lasso_mse), Lasso_r2]

## RandomForestRegression 

In [155]:
RFR = RandomForestRegressor(n_estimators=150)
RFR.fit(X_train,Y_train)

RFR_predict = RFR.predict(X_test)
RFR_mape = mean_absolute_percentage_error(Y_test,RFR_predict)
RFR_mse = mean_squared_error(Y_test,RFR_predict)
RFR_r2 = r2_score(Y_test,RFR_predict)
print('Mean Absolute Percentage Error for RFR is: ', RFR_mape)
print('Mean Squared Error for RFR is: ', sqrt(RFR_mse))
print('R2 score for RFR is: ', RFR_r2)

Mean Absolute Percentage Error for RFR is:  0.055707748845737974
Mean Squared Error for RFR is:  108311.23237392152
R2 score for RFR is:  0.8498620531647838


In [156]:
result['RFR'] = [RFR_mape, sqrt(RFR_mse), RFR_r2]

## SVR 

In [157]:
SVR_linear = SVR(kernel='linear')
SVR_linear.fit(X_train,Y_train)

SVR_predict = SVR_linear.predict(X_test)

SVR_predict

array([552635.9524333 , 455041.38812504, 468816.27321773, ...,
       531802.36625867, 708505.35436297, 647361.78417037])

In [158]:
SVR_mape = mean_absolute_percentage_error(Y_test,SVR_predict)
SVR_mse = mean_squared_error(Y_test,SVR_predict)
SVR_r2 = r2_score(Y_test,SVR_predict)
print('Mean Absolute Percentage Error for SVR is: ', SVR_mape)
print('Mean Squared Error for SVR is: ', sqrt(SVR_mse))
print('R2 score for SVR is: ', SVR_r2)

Mean Absolute Percentage Error for SVR is:  0.13340907190529233
Mean Squared Error for SVR is:  183519.43152122686
R2 score for SVR is:  0.568969910652777


In [159]:
result['SVR'] = [SVR_mape, sqrt(SVR_mse), SVR_r2]

## Result 

In [160]:
result

,Linear Regression,Ridge Regression,LASSO,RFR,SVR
MAPE,0.14,0.14,0.14,0.06,0.13
MSE,157998.43,157914.68,157997.79,108311.23,183519.43
r2 score,0.68,0.68,0.68,0.85,0.57
